<a href="https://colab.research.google.com/github/sfarrukhm/huggingface-learning/blob/main/finetuning_without_Trainer_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets=load_dataset("glue", "mrpc")
checkpoint="bert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets=raw_datasets.map(tokenize_function, batched=True)
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

#### Prepare data for training
Define dataloaders to iterate over batches

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [ ]:
# first we need to apply a bit of postprocessing to our tokenized_datasets
# to train the model, we only need following columns:
# 'input_ids', 'token_type_ids', 'attention_mask', 'labels'
# So we well remove not required columns and rename 'label' to 'labels'
tokenized_datasets=tokenized_datasets.remove_columns(['sentence1','sentence2','idx'])
tokenized_datasets=tokenized_datasets.rename_column("label","labels")

# we also need to set the format of the dataset to Pytorch tensor
tokenized_datasets.set_format("torch")


In [ ]:
# Now define dataloaders
from torch.utils.data import DataLoader
train_dataloader=DataLoader(tokenized_datasets['train'],shuffle=True,batch_size=8,
                            collate_fn=data_collator)
eval_dataloader=DataLoader(tokenized_datasets['validation'],shuffle=True, batch_size=8,
                           collate_fn=data_collator)
for batch in train_dataloader:
    break
{k:v.shape for k,v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 78]),
 'token_type_ids': torch.Size([8, 78]),
 'attention_mask': torch.Size([8, 78])}

#### Prepare the objects for training



1. Model

In [ ]:
from transformers import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
outputs=model(**batch)
# outputs.hidden_states

In [ ]:
outputs

SequenceClassifierOutput(loss=tensor(0.6148, grad_fn=<NllLossBackward0>), logits=tensor([[-0.0566,  0.3437],
        [-0.0636,  0.3337],
        [-0.0555,  0.3257],
        [-0.0491,  0.3323],
        [-0.0398,  0.3552],
        [-0.0600,  0.3411],
        [-0.0419,  0.3536],
        [-0.0659,  0.3297]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

All the 🤗 Transformers models return `loss` when `labels` are porvided along with `logits` for each input sample.

2. Define the optimizer:
We will se AdamW optimizer which also the default optimizer for `Trainer`. It is same as Adam but with a twist for weight decay regularization.
3. Define the learning rate scheduler:
`Trainer` implements lr as linearly decaying by default. To define it as linearly decaying, we would need number of training steps.

In [ ]:
from transformers import AdamW
optimizer=AdamW(model.parameters(),lr=5e-5)

# defining learning rate schadualer:
from transformers import get_scheduler
num_epochs=3
num_training_steps=num_epochs*len(train_dataloader)
lr_scheduler=get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


4. Define the training loop

In [ ]:
import torch
device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch={k: v.to(device) for k, v in batch.items()}
    outputs=model(**batch)
    loss = outputs.loss

    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

### The evaluation loop

In [ ]:
import evaluate
metric=evaluate.load("glue","mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
      ouputs=model(**batch)
    logits=ouputs.logits
    predictions=torch.argmax(logits,dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()
